In [1]:
import torch
import numpy as np
import pandas as pd
from itertools import cycle
from torch_geometric.data import Data
from src.data_preparation import DataPreprocessor
from src.graph_data_loader import graph_loader

In [2]:
data = graph_loader()
data

Data(x=[10122, 20], edge_index=[2, 46832])

In [7]:
print("Number of nodes: ", data.num_nodes)
print("Number of features per node: ", data.num_node_features)
print("Number of edges: ", data.num_edges)
print("Number of edge attributes: ", data.num_edge_features)
print("Undirected: ", data.is_undirected())
print("Isolated Nodes: ", data.has_isolated_nodes())
print("Self Loops: ", data.has_self_loops())

Number of nodes:  10122
Number of features per node:  20
Number of edges:  46832
Number of edge attributes:  0
Undirected:  True
Isolated Nodes:  True
Self Loops:  False
